In this project, we utilize certain segments of code from the Udacity Deep Learning Specialization to estimate word embeddings using the Word2Vec algorithm. This is permissible under the terms of the MIT License associated with the original source code. 

To optimize the performance for our specific dataset, we've made minor adjustments to the parameters' setup within the model. In addition, we have also tailored the pre-processing steps to better suit our data. The data we employ consists of pre-processed newspaper articles from four major German news sources - Süddeutsche Zeitung (SZ), Handelsblatt (HB), Deutsche Presse-Agentur (dpa), and Die Welt.

# Skip-gram Word2Vec

---
## Word embeddings

When you're dealing with words in text, you end up with tens of thousands of word classes to analyze; one for each word in a vocabulary. Trying to one-hot encode these words is massively inefficient because most values in a one-hot vector will be set to zero. So, the matrix multiplication that happens in between a one-hot input vector and a first, hidden layer will result in mostly zero-valued hidden outputs.

To solve this problem and greatly increase the efficiency of our networks, we use what are called **embeddings**. Embeddings are just a fully connected layer. We call this layer the embedding layer and the weights are embedding weights. We skip the multiplication into the embedding layer by instead directly grabbing the hidden layer values from the weight matrix. We can do this because the multiplication of a one-hot encoded vector with a matrix returns the row of the matrix corresponding to the index of the "on" input unit.

<img src='assets/lookup_matrix.png' width=50%>

Instead of doing the matrix multiplication, we use the weight matrix as a lookup table. We encode the words as integers, for example "heart" is encoded as 958, "mind" as 18094. Then to get hidden layer values for "heart", we just take the 958th row of the embedding matrix. This process is called an **embedding lookup** and the number of hidden units is the **embedding dimension**.
 
There is nothing magical going on here. The embedding lookup table is just a weight matrix. The embedding layer is just a hidden layer. The lookup is just a shortcut for the matrix multiplication. The lookup table is trained just like any weight matrix.

Embeddings aren't only used for words of course. We can use them for any model where we have a massive number of classes. A particular type of model called **Word2Vec** uses the embedding layer to find vector representations of words that contain semantic meaning.

---
## Word2Vec

The Word2Vec algorithm finds much more efficient representations by finding vectors that represent the words. These vectors also contain semantic information about the words.

<img src="assets/context_drink.png" width=40%>

Words that show up in similar **contexts**, such as "coffee", "tea", and "water" will have vectors near each other. Different words will be further away from one another, and relationships can be represented by distance in vector space.


There are two architectures for implementing Word2Vec:
>* CBOW (Continuous Bag-Of-Words) and 
* Skip-gram

<img src="assets/word2vec_architectures.png" width=60%>

In this implementation, we'll be using the **skip-gram architecture** with **negative sampling** because it performs better than CBOW and trains faster with negative sampling. Here, we pass in a word and try to predict the words surrounding it in the text. In this way, we can train the network to learn representations for words that show up in similar contexts.

---
## Loading Data

Next, we load the pre-processed datasets for Handelsblatt, SZ, Welt, and dpa.

In [1]:
import os
import pandas as pd
from ast import literal_eval

# Set the path variable to point to the 'newspaper_data_processing' directory, which is located in the same 
# parent directory as 'newspaper_analysis'. This is the repository used for data pre-processing.
path = os.getcwd().replace('\\newspaper_analysis\\word_embeddings', '') + '\\newspaper_data_processing'

# Load pre-processed 'dpa' dataset from a CSV file.
dpa = pd.read_csv(path + '\\dpa\\' + 'dpa_prepro_final.csv', encoding = 'utf-8', sep=';', index_col = 0,  keep_default_na=False,
                   dtype = {'rubrics': 'str', 
                            'source': 'str',
                            'keywords': 'str',
                            'title': 'str',
                            'city': 'str',
                            'genre': 'str',
                            'wordcount': 'str'},
                  converters = {'paragraphs': literal_eval})

# Keep only the article texts and their respective publication dates.
dpa = dpa[['texts', 'day', 'month', 'year']]

# Load pre-processed 'SZ' dataset from a CSV file.
sz = pd.read_csv(path + '\\SZ\\' + 'sz_prepro_final.csv', encoding = 'utf-8-sig', sep=';', index_col = 0, dtype = {'newspaper': 'str',
                                                                                                 'newspaper_2': 'str',
                                                                                                 'quelle_texts': 'str',
                                                                                                 'page': 'str',
                                                                                                 'rubrics': 'str'})
sz.page = sz.page.fillna('')
sz.newspaper = sz.newspaper.fillna('')
sz.newspaper_2 = sz.newspaper_2.fillna('')
sz.rubrics = sz.rubrics.fillna('')
sz.quelle_texts = sz.quelle_texts.fillna('')

# Keep only the article texts and their respective publication dates.
sz = sz[['texts', 'day', 'month', 'year']]

# Load pre-processed 'Handelsblatt' dataset from a CSV file.
hb = pd.read_csv(path + '\\Handelsblatt\\' + 'hb_prepro_final.csv', encoding = 'utf-8-sig', sep=';', index_col = 0, dtype = {'kicker': 'str',
                                                                                                 'page': 'str',
                                                                                                 'series_title': 'str',
                                                                                                 'rubrics': 'str'})
hb.page = hb.page.fillna('')
hb.series_title = hb.series_title.fillna('')
hb.kicker = hb.kicker.fillna('')
hb.rubrics = hb.rubrics.fillna('')

# Keep only the article texts and their respective publication dates.
hb = hb[['texts', 'day', 'month', 'year']]

# Load pre-processed 'Welt' dataset from a CSV file.
welt = pd.read_csv(path + '\\Welt\\' + 'welt_prepro_final.csv', encoding = 'utf-8-sig', sep=';', index_col = 0, dtype = {'newspaper': 'str',
                                                                                                 'rubrics': 'str',
                                                                                                 'title': 'str'})
welt.title = welt.title.fillna('')
welt.rubrics = welt.rubrics.fillna('')

# Keep only the article texts and their respective publication dates.
welt = welt[['texts', 'day', 'month', 'year']]

# Concatenate the 'dpa', 'sz', 'hb', and 'welt' DataFrames into a single DataFrame 'data'.
data = pd.concat([dpa, sz, hb, welt])

# The number of articles in the final dataset.
print(len(data))

# Sort the data in chronological order.
data = data.sort_values(['year', 'month', 'day'], ascending=[True, True, True])
# Reset the index of the DataFrame
data.reset_index(inplace=True, drop=True)
data.head()

3336299


,texts,day,month,year
0,Schalck: Milliardenkredit sicherte Zahlungsfäh...,1,1,1991
1,Welajati: Iran bleibt bei einem Krieg am Golf ...,1,1,1991
2,Bush will offenbar seinen Außenminister erneut...,1,1,1991
3,Sperrfrist 1. Januar 1000 HBV fordert umfassen...,1,1,1991
4,Schamir weist Nahost-Äußerungen des neuen EG-P...,1,1,1991


In this project, we limit our training dataset to include only articles published from 1991 through 2009, inclusive. This decision is driven by our later goal of performing an out-of-sample forecasting exercise for GDP growth covering the years 2010 to 2018. By excluding data from 2010 onward during the model training phase, we ensure that our forecast does not incorporate information that was not yet available.

In [2]:
# Filter the dataset to include only articles published before 2010.
data = data[data.year < 2010]

For the subsequent steps, we will only require the text content of the articles, disregarding their publication date. Therefore, we create `data_texts`, a list that contains the text of all the articles.

In [3]:
data_texts = list(data.texts)

## Pre-processing

In this section, we're applying some pre-processing steps to our text data to facilitate the training of our Word2Vec model. This is achieved by our `preprocess` function, which performs several tasks:

* It transforms all characters in the text to lowercase. This helps to ensure that the same word in different cases is not treated as different words.
* It removes all punctuation from the text. The rationale behind this is that punctuation often doesn't carry significant semantic meaning and can add noise to the model.
* It removes all non-alphabetic characters. This step reduces noise in the data and focuses the model on the words themselves.
* It replaces multiple whitespaces with a single space, ensuring a clean and consistent tokenization process.
* It tokenizes the text, i.e., it breaks the text down into a list of individual words, which is the input our Word2Vec model requires.
* It removes single-letter tokens because they do not carry significant meaning or context.

In [4]:
from datetime import datetime
startTime = datetime.now()

import multiprocessing as mp
# Set to the number of cores you want to use
NUM_CORE = mp.cpu_count()-8

import preprocess

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    list_of_tokens = pool.map(preprocess.preprocess, [text for text in data_texts]) 
    pool.close()
    pool.join()
    
print(datetime.now() - startTime)

0:03:46.533147


Additionally, we remove all words that show up five or *fewer* times in the dataset. This will greatly reduce issues due to noise in the data and improve the quality of the vector representations. 

In [5]:
import itertools
# Get list of words
words = list(itertools.chain.from_iterable(list_of_tokens))

from collections import Counter
word_counts = Counter(words)
# Remove all words with 5 or fewer occurences
words = [word for word in words if word_counts[word] > 5]
print(words[:100])

['schalck', 'milliardenkredit', 'sicherte', 'zahlungsfähigkeit', 'der', 'exddr', 'berlin', 'nach', 'darstellung', 'des', 'früheren', 'devisenbeschaffers', 'der', 'ehemaligen', 'ddr', 'alexander', 'schalckgolodkowski', 'hat', 'der', 'von', 'franzjosef', 'strauß', 'eingefädelte', 'milliardenkredit', 'von', 'erstmals', 'die', 'zahlungsfähigkeit', 'der', 'ddr', 'aufrechterhalten', 'in', 'einem', 'interview', 'der', 'ard', 'das', 'am', 'mittwoch', 'abend', 'ausgestrahlt', 'wird', 'sagte', 'schalck', 'nach', 'angaben', 'des', 'senders', 'freies', 'berlin', 'sfb', 'vom', 'dienstag', 'damals', 'sei', 'es', 'bereits', 'um', 'sein', 'oder', 'nichtsein', 'der', 'ddr', 'gegangen', 'ende', 'sei', 'dann', 'deutlich', 'geworden', 'daß', 'die', 'damalige', 'ddr', 'nur', 'noch', 'in', 'enger', 'wirtschaftlicher', 'kooperation', 'mit', 'der', 'bundesrepublik', 'überleben', 'könnte', 'dies', 'sei', 'aber', 'von', 'staats', 'und', 'parteichef', 'erich', 'honecker', 'und', 'seinen', 'beratern', 'als', 'kon

In [6]:
# Print some stats about this word data
print("Total words in text: {}".format(len(words)))
print("Unique words: {}".format(len(set(words)))) # `set` removes any duplicate words

Total words in text: 641418676
Unique words: 757990


### Dictionaries

Next, we are creating two dictionaries to convert words to integers and back again (integers to words). This is done with a function in the `create_lookup_tables.py` file. `create_lookup_tables` takes in a list of words in a text and returns two dictionaries.
>* The integers are assigned in descending frequency order, so the most frequent word ("der") is given the integer 0 and the next most frequent is 1, and so on. 

Once we have our dictionaries, the words are converted to integers and stored in the list `int_words`.

In [7]:
import create_lookup_tables

vocab_to_int, int_to_vocab = create_lookup_tables.create_lookup_tables(words)
int_words = [vocab_to_int[word] for word in words]

print(int_words[:30])

[19334, 30897, 6490, 26851, 0, 18958, 95, 19, 1714, 12, 522, 178459, 0, 733, 1598, 2383, 27781, 31, 0, 5, 14896, 6854, 80484, 30897, 5, 406, 1, 26851, 0, 1598]


## Subsampling

Words that show up often such as "der", "die", and "in" don't provide much context to the nearby words. If we discard some of them, we can remove some of the noise from our data and in return get faster training and better representations. This process is called subsampling by Mikolov. For each word $w_i$ in the training set, we'll discard it with probability given by 

$$ P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}} $$

where $t$ is a threshold parameter and $f(w_i)$ is the frequency of word $w_i$ in the total dataset.

Next, we implement subsampling for the words in `int_words`. That is, we go through `int_words` and discard each word given the probablility $P(w_i)$ shown above. Note that $P(w_i)$ is the probability that a word is discarded. We assign the subsampled data to `train_words`.

In [8]:
import random
import numpy as np

threshold = 1e-5
# Dictionary of int_words, how many times they appear
word_counts = Counter(int_words)

total_count = len(int_words)
freqs = {word: count/total_count for word, count in word_counts.items()}
p_drop = {word: 1 - np.sqrt(threshold/freqs[word]) for word in word_counts}
# Discard some frequent words, according to the subsampling equation
# Create a new list of words for training
train_words = [word for word in int_words if random.random() < (1 - p_drop[word])]

print(train_words[:30])

# The number of words in all the texts
print(len(train_words))

[19334, 30897, 6490, 26851, 18958, 1714, 178459, 1598, 2383, 27781, 14896, 6854, 80484, 30897, 26851, 9380, 14570, 19334, 129, 8092, 12334, 43108, 91, 88758, 1598, 115, 3142, 5360, 954, 3006]
213433705


## Making batches

Now that our data is in good shape, we need to get it into the proper form to pass it into our network. With the skip-gram architecture, for each word in the text, we want to define a surrounding _context_ and grab all the words in a window around that word, with size $C$. 

From [Mikolov et al.](https://arxiv.org/pdf/1301.3781.pdf): 

"Since the more distant words are usually less related to the current word than those close to it, we give less weight to the distant words by sampling less from those words in our training examples... If we choose $C = 5$, for each training word we will select randomly a number $R$ in range $[ 1: C ]$, and then use $R$ words from history and $R$ words from the future of the current word as correct labels."

We implement a function `get_target` that receives a list of words, an index, and a window size, then returns a list of words in the window around the index. We use the algorithm described above, where we chose a random number of words from the window to use as context words.

Say, we have an input and we're interested in the idx=2 token, `741`: 
```
[5233, 58, 741, 10571, 27349, 0, 15067, 58112, 3580, 58, 10712]
```

For `R=2`, `get_target` will return a list of four values:
```
[5233, 58, 10571, 27349]
```

In [9]:
def get_target(words, idx, window_size=10):
    ''' Get a list of words in a window around an index. '''
    
    R = np.random.randint(1, window_size+1)
    start = idx - R if (idx - R) > 0 else 0
    stop = idx + R
    target_words = words[start:idx] + words[idx+1:stop+1]
    
    return list(target_words)

In [10]:
# Run this cell multiple times to check for random window selection
int_text = [i for i in range(10)]
print('Input: ', int_text)
idx = 5 # Word index of interest

target = get_target(int_text, idx=idx, window_size=5)
print('Target: ', target)  # You should get some indices around the idx

Input:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Target:  [3, 4, 6, 7]


### Generating Batches 

Here's a generator function that returns batches of input and target data for our model, using the `get_target` function from above. The idea is that it grabs `batch_size` words from a words list. Then for each of those batches, it gets the target words in a window.

In [11]:
def get_batches(words, batch_size, window_size=10):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    
    n_batches = len(words)//batch_size
    
    # Only full batches
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y    

In [12]:
int_text = [i for i in range(20)]
x,y = next(get_batches(int_text, batch_size=4, window_size=10))

print('x\n', x)
print('y\n', y)

x
 [0, 1, 1, 1, 2, 2, 2, 3, 3, 3]
y
 [1, 0, 2, 3, 0, 1, 3, 0, 1, 2]


---
## Validation

Here, we are creating a function that will help us observe our model as it learns. We're going to choose a few common words and few uncommon words. Then, we'll print out the closest words to them using the cosine similarity: 

<img src="assets/two_vectors.png" width=30%>

$$
\mathrm{similarity} = \cos(\theta) = \frac{\vec{a} \cdot \vec{b}}{|\vec{a}||\vec{b}|}
$$


We can encode the validation words as vectors $\vec{a}$ using the embedding table, then calculate the similarity with each word vector $\vec{b}$ in the embedding table. With the similarities, we can print out the validation words and words in our embedding table semantically similar to those words. It's a nice way to check that our embedding table is grouping together words with similar semantic meanings.

In [13]:
def cosine_similarity(embedding, valid_size=16, valid_window=300, device='cpu'):
    """ Returns the cosine similarity of validation words with words in the embedding matrix.
        Here, embedding should be a PyTorch embedding module.
    """
    
    # Here we're calculating the cosine similarity between some random words and 
    # our embedding vectors. With the similarities, we can look at what words are
    # close to our random words.
    
    # sim = (a . b) / |a||b|
    
    embed_vectors = embedding.weight
    
    # Magnitude of embedding vectors, |b|
    magnitudes = embed_vectors.pow(2).sum(dim=1).sqrt().unsqueeze(0)
    
    # Pick N words from our ranges (0,window) and (5000,5000+window). lower id implies more frequent 
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples,
                               random.sample(range(5000,5000+valid_window), valid_size//2))
    valid_examples = torch.LongTensor(valid_examples).to(device)
    
    valid_vectors = embedding(valid_examples)
    similarities = torch.mm(valid_vectors, embed_vectors.t())/magnitudes
        
    return valid_examples, similarities

---
# SkipGram model

Here we define and train the SkipGram model. We'll define an [embedding layer](https://pytorch.org/docs/stable/nn.html#embedding) and a final, softmax output layer.

An Embedding layer takes in a number of inputs, importantly:
* **num_embeddings** – the size of the dictionary of embeddings, or how many rows you'll want in the embedding weight matrix
* **embedding_dim** – the size of each embedding vector; the embedding dimension

Below is an approximate diagram of the general structure of our network.
<img src="assets/skip_gram_arch.png" width=60%>

* The input words are passed in as batches of input word tokens. 
* This will go into a hidden layer of linear units (our embedding layer). 
* Then, finally into a softmax output layer. 

We'll use the softmax layer to make a prediction about the context words by sampling, as usual.

---
## Negative Sampling

For every example we give the network, we train it using the output from the softmax layer. That means for each input, we're making very small changes to millions of weights even though we only have one true example. This makes training the network very inefficient. We can approximate the loss from the softmax layer by only updating a small subset of all the weights at once. We'll update the weights for the correct example, but only a small number of incorrect, or noise, examples. This is called ["negative sampling"](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf). 

There are two modifications we need to make. First, since we're not taking the softmax output over all the words, we're really only concerned with one output word at a time. Similar to how we use an embedding table to map the input word to the hidden layer, we can now use another embedding table to map the hidden layer to the output word. Now we have two embedding layers, one for input words and one for output words. Secondly, we use a modified loss function where we only care about the true example and a small subset of noise examples.

$$
- \large \log{\sigma\left(u_{w_O}\hspace{0.001em}^\top v_{w_I}\right)} -
\sum_i^N \mathbb{E}_{w_i \sim P_n(w)}\log{\sigma\left(-u_{w_i}\hspace{0.001em}^\top v_{w_I}\right)}
$$

This is a little complicated so we'll go through it bit by bit. $u_{w_O}\hspace{0.001em}^\top$ is the embedding vector for our "output" target word (transposed, that's the $^\top$ symbol) and $v_{w_I}$ is the embedding vector for the "input" word. Then the first term 

$$\large \log{\sigma\left(u_{w_O}\hspace{0.001em}^\top v_{w_I}\right)}$$

says we take the log-sigmoid of the inner product of the output word vector and the input word vector. Now the second term, let's first look at 

$$\large \sum_i^N \mathbb{E}_{w_i \sim P_n(w)}$$ 

This means we're going to take a sum over words $w_i$ drawn from a noise distribution $w_i \sim P_n(w)$. The noise distribution is basically our vocabulary of words that aren't in the context of our input word. In effect, we can randomly sample words from our vocabulary to get these words. $P_n(w)$ is an arbitrary probability distribution though, which means we get to decide how to weight the words that we're sampling. This could be a uniform distribution, where we sample all words with equal probability. Or it could be according to the frequency that each word shows up in our text corpus, the unigram distribution $U(w)$. The authors found the best distribution to be $U(w)^{3/4}$, empirically. 

Finally, in 

$$\large \log{\sigma\left(-u_{w_i}\hspace{0.001em}^\top v_{w_I}\right)},$$ 

we take the log-sigmoid of the negated inner product of a noise vector with the input vector. 

<img src="assets/neg_sampling_loss.png" width=50%>

To give you an intuition for what we're doing here, remember that the sigmoid function returns a probability between 0 and 1. The first term in the loss pushes the probability that our network will predict the correct word $w_O$ towards 1. In the second term, since we are negating the sigmoid input, we're pushing the probabilities of the noise words towards 0.

In [14]:
import torch
from torch import nn
import torch.optim as optim

In [15]:
class SkipGramNeg(nn.Module):
    def __init__(self, n_vocab, n_embed, noise_dist=None):
        super().__init__()
        
        self.n_vocab = n_vocab
        self.n_embed = n_embed
        self.noise_dist = noise_dist
        
        # Define embedding layers for input and output words
        self.in_embed = nn.Embedding(n_vocab, n_embed)
        self.out_embed = nn.Embedding(n_vocab, n_embed)
        
        # Initialize embedding tables with uniform distribution
        # (this helps with convergence)
        self.in_embed.weight.data.uniform_(-1, 1)
        self.out_embed.weight.data.uniform_(-1, 1)
        
    def forward_input(self, input_words):
        input_vectors = self.in_embed(input_words)
        return input_vectors
    
    def forward_output(self, output_words):
        output_vectors = self.out_embed(output_words)
        return output_vectors
    
    def forward_noise(self, batch_size, n_samples):
        """ Generate noise vectors with shape (batch_size, n_samples, n_embed)"""
        if self.noise_dist is None:
            # Sample words uniformly
            noise_dist = torch.ones(self.n_vocab)
        else:
            noise_dist = self.noise_dist
            
        # Sample words from our noise distribution
        noise_words = torch.multinomial(noise_dist,
                                        batch_size * n_samples,
                                        replacement=True)
        
        device = "cuda" if model.out_embed.weight.is_cuda else "cpu"
        noise_words = noise_words.to(device)
        
        noise_vectors = self.out_embed(noise_words).view(batch_size, n_samples, self.n_embed)
        
        return noise_vectors

In [16]:
class NegativeSamplingLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, input_vectors, output_vectors, noise_vectors):
        
        batch_size, embed_size = input_vectors.shape
        
        # Input vectors should be a batch of column vectors
        input_vectors = input_vectors.view(batch_size, embed_size, 1)
        
        # Output vectors should be a batch of row vectors
        output_vectors = output_vectors.view(batch_size, 1, embed_size)
        
        # bmm = batch matrix multiplication
        # Correct log-sigmoid loss
        out_loss = torch.bmm(output_vectors, input_vectors).sigmoid().log()
        out_loss = out_loss.squeeze()
        
        # Incorrect log-sigmoid loss
        noise_loss = torch.bmm(noise_vectors.neg(), input_vectors).sigmoid().log()
        noise_loss = noise_loss.squeeze().sum(1)  # sum the losses over the sample of noise vectors

        # Negate and sum correct and noisy log-sigmoid losses
        # Return average batch loss
        return -(out_loss + noise_loss).mean()

### Training

Below is our training loop, and we train the model on GPU (NVIDIA GeForce RTX 2080 Ti).

In [17]:
startTime = datetime.now()

import codecs

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Get our noise distribution using word frequencies calculated earlier in the notebook
word_freqs = np.array(sorted(freqs.values(), reverse=True))
unigram_dist = word_freqs/word_freqs.sum()
noise_dist = torch.from_numpy(unigram_dist**(0.75)/np.sum(unigram_dist**(0.75)))

# Instantiating the model
embedding_dim = 256
model = SkipGramNeg(len(vocab_to_int), embedding_dim, noise_dist=noise_dist).to(device)

# Using the loss that we defined
criterion = NegativeSamplingLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.0001)

print_every = 30000
steps = 0
epochs = 10

# Train for some number of epochs
for e in range(epochs):
    
    # Get our input, target batches
    for input_words, target_words in get_batches(train_words, 128):
        steps += 1
        inputs, targets = torch.LongTensor(input_words), torch.LongTensor(target_words)
        inputs, targets = inputs.to(device), targets.to(device)
        
        # Input, output, and noise vectors
        input_vectors = model.forward_input(inputs)
        output_vectors = model.forward_output(targets)
        noise_vectors = model.forward_noise(inputs.shape[0], 5)

        # Negative sampling loss
        loss = criterion(input_vectors, output_vectors, noise_vectors)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Loss stats
        if steps % print_every == 0:
            print("Epoch: {}/{}".format(e+1, epochs))
            print("Loss: ", loss.item()) # Avg batch loss at this point in training
            valid_examples, valid_similarities = cosine_similarity(model.in_embed.to('cpu'), device='cpu')
            _, closest_idxs = valid_similarities.topk(6)

            valid_examples, closest_idxs = valid_examples.to('cpu'), closest_idxs.to('cpu')
            for ii, valid_idx in enumerate(valid_examples):
                closest_words = [int_to_vocab[idx.item()] for idx in closest_idxs[ii]][1:]
                print(int_to_vocab[valid_idx.item()] + " | " + ', '.join(closest_words))
            print("...\n")
            
            model.in_embed.to(device)  # Moved back to GPU
            
    # Saving embeddings after each epoch
    print("Saving embeddings after epoch: {}/{}".format(e+1, epochs))
    
    # Getting embeddings from the embedding layer of our model, by name
    embeddings = model.in_embed.weight.to('cpu').data.numpy()

    # Create a list of vocabulary words corresponding to each ID
    vocab = [int_to_vocab[i] for i in range(len(embeddings))]

    # Create a list of IDs corresponding to each word 
    id_list = [i for i in range(len(embeddings))]

    # Create a dictionary where each word (key) is associated with its corresponding embedding vector (value)
    word_embed = dict(zip(vocab, embeddings))

    # Create a dictionary where each ID (key) is associated with its corresponding word (value)
    id_word = dict(zip(id_list, vocab))
    
    with codecs.open("news_word2vec.txt", "w", "utf-8") as f:
        for id, word in id_word.items():
            vec = embeddings[id, :].tolist()
            vec = [np.round(num, 6) for num in vec]
            vec_str = " ".join(map(str, vec))
            f.write("%s %s\n" % (word, vec_str))
            
    # Move the model back to GPU for further training
    model.to(device)
    
    print(datetime.now()-startTime)

Epoch: 1/10
Loss:  10.164682388305664
seiner | kinderfreibetrag, gealterter, migas, neugebaute, akumu
wird | usbuchgruppe, formeleins, nóbrega, prozeßtechnologien, hollywoodmilliardär
partei | ausreisegenehmigung, geldard, eustace, primärenergien, ephraïm
nun | pdsobfrau, relax, romasiedlungen, regierungsbeamtin, programmformen
anderen | entlaubt, rheinruhrzentrum, anstinken, unpolizist, zeitgemäßes
während | arbeitsplatzperspektiven, tunisie, lehrerkollegiums, bundestagssitzung, mitarbeiterbetrieb
schröder | interessenssphären, rupprechtnachfolge, schatzministers, unlenkbar, speiserestaurant
wegen | preisen, sadcgipfel, briefverteil, pkoaktien, liquiditätskonten
erneuerung | spionageflugzeugs, hysterisierung, bogenschütz, highdefinition, steuergünstigeren
dirk | bushmännern, ussteinkohleproduzenten, parteiordnungsverfahren, ponyreiten, verschrauben
formen | genesung, auseinanderdriftenden, idlout, zentralisierter, funkstreife
zügig | fertigteilwerk, kasse, miniaturformat, wallende, ex

Epoch: 1/10
Loss:  3.8655049800872803
markt | auf, um, aufgrund, rund, dagegen
zahl | rund, jahr, ostdeutschland, insgesamt, im
erklärte | es, sagte, daß, nicht, keine
zuvor | nach, sich, donnerstag, wurde, einem
ab | sich, der, mit, von, bei
politik | nicht, wenn, müsse, eine, diese
sagte | daß, er, es, seine, alle
könne | nicht, es, sei, eine, werden
dirk | geteilt, lockerung, krisensituationen, gesehen, attraktivsten
preiserhöhungen | zusätzlich, köln, hamburg, maßnahmen, schaffen
damaskus | israel, treffen, besetzten, gespräche, washington
läden | jeden, fernsehen, band, zumindest, rpt
lohnfortzahlung | begleitprogramm, bogota, unterdessen, verschärften, metall
bundeshaushalt | vereinigung, ost, weniger, tun, ostdeutschland
nötige | früherer, französischen, hören, neuer, europa
ursula | gesamten, deutschen, europa, mark, vorjahr
...

Epoch: 1/10
Loss:  4.22331428527832
fast | im, von, während, etwa, bei
hieß | am, nach, war, in, den
angaben | worden, war, zwei, in, am
anderen | auc

Epoch: 1/10
Loss:  2.9975619316101074
hat | dem, der, in, den, und
auch | wie, zu, die, aber, ist
dieses | sieht, damit, durch, für, wird
dann | nicht, auch, aber, eine, wie
muss | dass, mag, ja, politik, wenn
damit | für, bei, diesem, lediglich, sieht
sollten | daß, könne, darauf, keine, zur
könnte | zu, eine, daß, auch, es
ausbruch | region, beobachter, ausgelöst, berichten, verantwortlich
finanzpolitik | wirtschafts, wirtschaft, arbeitslosigkeit, reformen, wirtschaftspolitik
mitsubishi | marktanteil, sinkenden, stiegen, motors, gewinne
technisch | beispiel, derzeit, bietet, somit, beispielsweise
friedensprozeß | israel, arafat, israels, jerusalem, frieden
irans | iranischen, iran, teheran, diplomaten, delegation
innen | öffentlichkeit, verantwortung, wort, willen, frage
besserung | anstieg, rückgang, niveau, westdeutschland, zunahme
...

Epoch: 1/10
Loss:  2.7605535984039307
regierung | unterstützung, politische, gewarnt, politischen, politiker
milliarden | mrd, dm, jahr, betragen, 

Epoch: 1/10
Loss:  3.3054637908935547
nicht | kein, sollte, dass, dann, es
dm | millionen, bezifferte, steuern, milliarden, milliarde
ihre | die, und, den, wie, ihren
sieht | für, dieses, allerdings, sowohl, kommen
in | vor, dem, einer, aus, und
weil | sie, selbst, nicht, lassen, kein
wurde | war, daraufhin, hatte, selben, später
es | zu, dass, ihrer, nicht, sie
erich | hamburger, köln, geschäftsführer, düsseldorfer, münchner
fleisch | rindfleisch, tiere, bauern, lebensmittel, bse
ursprünglichen | voraussichtlich, werde, bisherigen, für, erfolgen
streben | künftige, betont, schaffen, bestehen, entscheidende
aufwendungen | steuern, dm, erträge, größenordnung, steigerung
panzer | armee, beschossen, getötet, soldaten, bewaffnete
vorsprung | gewonnen, vorn, rennen, gewinnen, gewinnt
strecken | fluggesellschaften, bahn, lufthansa, betrieb, kleinere
...

Epoch: 1/10
Loss:  2.628655195236206
spd | grünen, fdp, csu, bundestag, cdu
jahr | rechnet, liegen, laufenden, drittel, hälfte
jahren | inz

Epoch: 1/10
Loss:  2.9116930961608887
darauf | dass, keine, könne, sollte, dies
wirtschaft | wirtschaftlichen, wirtschaftliche, veränderung, zeige, stärkere
mittwoch | donnerstag, dienstag, montag, freitag, am
sie | lassen, sondern, selbst, es, jeden
banken | kapitalmarkt, kreditinstitute, großbanken, institute, kredite
des | der, einer, dem, vor, vom
gehen | auch, noch, kommen, geht, kommt
keine | keinen, dass, nicht, eine, sollte
vorsprung | vorn, gewonnen, gewinnt, stärkste, abschneiden
ursprünglichen | bisherigen, erfolgen, ursprünglich, plänen, ab
holzmann | pleite, schneider, berger, baukonzern, unternehmens
bundeskabinett | gesetzentwurf, bundesregierung, bundesrat, arbeitsgruppe, beschlossenen
großbritanniens | frankreichs, gipfeltreffen, gipfel, premierminister, washingtons
asset | investment, fondsmanager, institutionelle, portfolio, fonds
euroraum | inflation, eurozone, inflationsrate, euroland, volkswirte
meldung | nachdem, meldungen, gehandelt, abgegeben, gab
...

Epoch: 1

...

Epoch: 10/10
Loss:  2.4694888591766357
will | soll, neue, wollen, wolle, neuen
ein | das, dem, einen, den, mit
hier | so, gerade, vielen, nur, beispiel
usa | vereinigten, amerikanische, amerikanischen, amerikanischer, washington
keine | keinen, es, nicht, darauf, ohne
beiden | beide, zusammen, dem, mit, unter
wurde | war, hatte, nach, dem, am
müsse | dürfe, könne, notwendig, nötig, gehe
beifall | applaus, redner, zuhörern, kämpferischen, rednerpult
überdies | begrenzt, dadurch, erschwert, betrifft, verbundenen
technisch | technischer, technische, technischen, technik, techniken
sat | rtl, kofler, premiere, dsf, prosieben
finanzpolitik | finanzpolitischen, haushaltspolitik, staatsfinanzen, haushaltskonsolidierung, wirtschaftspolitischen
ökonomischen | ökonomische, ökonomischer, ökonomie, wirtschaftliches, wirtschaftlicher
effekt | effekte, tendenziell, niedrige, bedingt, niedrigen
befreiung | besatzungsmacht, bewaffneten, befreiten, opfern, armee
...

Epoch: 10/10
Loss:  2.5875101089477

Epoch: 10/10
Loss:  2.364908456802368
sei | habe, keine, sagte, könne, werde
kommenden | nächsten, voraussichtlich, kommende, erwartet, diesem
nun | jetzt, steht, erst, schon, noch
lassen | dann, wenn, selbst, weil, nicht
dass | aber, nicht, deshalb, dieser, wenn
dollar | usdollar, yen, pfund, cents, us
damit | allerdings, für, dieses, einen, bei
hieß | zunächst, gegeben, weitere, weiteren, zufolge
ariel | scharon, scharons, ehud, barak, netanjahu
irans | teherans, teheran, iran, iranische, iranischen
strecken | streckennetz, strecke, regionalverkehr, langstrecken, flughäfen
löste | ausgelöst, verhinderte, auslöste, folgte, gestürzt
alan | greenspan, usnotenbankchef, fedchef, greenspans, bernanke
zerstörung | zerstören, zerstört, zerstörungen, vernichtung, menschenleben
tiere | tieren, rinder, schweine, tier, rindern
hypo | estate, depfa, vereinsbank, hypovereinsbank, commerzbank
...

Epoch: 10/10
Loss:  2.8955750465393066
teil | etwa, dabei, aus, außerdem, zwischen
partei | liberalen,

Epoch: 10/10
Loss:  2.3916642665863037
müssen | können, deshalb, muss, wenn, dafür
donnerstag | mittwoch, dienstag, freitag, montag, am
fast | während, sogar, vergangenen, letzten, seit
uns | wir, man, unsere, unser, alles
nun | jetzt, steht, erst, schon, wird
millionen | rund, jahr, insgesamt, milliarden, knapp
ihre | ihren, ihrer, anderen, haben, die
könne | müsse, möglich, darauf, gehe, gebe
wahlgang | stichwahl, amtsinhaber, gegenkandidaten, gegenkandidat, wahlgängen
beschleunigt | beschleunigte, beschleunigten, gebremst, beschleunigen, verlangsamt
bundeskartellamt | kartellamt, bundeskartellamts, bundeskartellamtes, kartellamts, kartellamtes
gehörenden | gehörende, tochtergesellschaft, tochterunternehmen, holding, übernommen
wen | wem, jemanden, niemanden, selber, wahr
erneuerung | programmatischen, akzente, programmatische, erneuern, aufbruch
südlichen | nördlichen, süden, südliche, südlich, norden
bewaffnete | bewaffneten, sicherheitskräfte, bewaffneter, sicherheitskräften, töte

Epoch: 10/10
Loss:  2.479637384414673
zur | eine, für, einer, über, dazu
fall | ob, verfahren, weder, gestellt, keine
ihr | sie, ihrem, selbst, ihren, ihrer
hier | so, gerade, beispiel, vielen, ganz
diesen | dieser, diese, alle, ohne, aber
sich | auch, den, zu, die, ein
haben | sich, zu, den, vor, die
im | einem, von, auf, nach, vor
einkommensteuer | körperschaftsteuer, bemessungsgrundlage, spitzensteuersatz, steuersätze, steuersatz
ursula | dgbvorsitzende, engelenkefer, dgbvize, bundesarbeitsminister, familienministerin
überdies | dadurch, begrenzt, erschwert, wären, betrifft
freitagabend | mittwochabend, dienstagabend, montagabend, donnerstagabend, abend
kalifornien | kalifornischen, illinois, kalifornische, usbundesstaat, oregon
dirk | ralf, christoph, uwe, claus, jens
mobilfunk | festnetz, mobilfunkmarkt, eplus, mobilfunkgeschäft, tmobile
konfrontation | entschlossenheit, konflikt, provozieren, isoliert, hardliner
...

Epoch: 10/10
Loss:  2.5237929821014404
war | wurde, damals, kam

Epoch: 10/10
Loss:  2.7075729370117188
wer | gar, einfach, wenn, jeder, egal
angaben | berichtete, teilte, worden, zufolge, mehreren
zur | eine, für, einer, über, dazu
wir | uns, unsere, unser, machen, man
sogar | weniger, fast, immerhin, dennoch, obwohl
die | den, der, und, zu, sich
anderen | andere, sind, ihre, diesen, dabei
sieht | stellt, für, dieses, hält, allerdings
neuem | zieht, rückt, neues, kehrt, bricht
gefasst | festgehalten, mutmaßlicher, mutmaßliche, mutmaßlichen, gefaßt
bomben | bombe, sprengsätze, explosionen, sprengstoff, granaten
organisierten | organisierte, organisierter, organisiert, organisationen, kriminalität
unbekannte | unbekannten, polizist, verübt, erschossen, polizeisprecher
präsidentschaft | regierungschefs, jacques, chirac, prodi, eukommissionspräsident
albaner | albanern, albanische, albanischen, kosovoalbaner, serben
bundeshaushalt | bundesetat, bundeshaushalts, etat, nettokreditaufnahme, steuermehreinnahmen
...

Epoch: 10/10
Loss:  2.6996684074401855
k

In [18]:
print(datetime.now()-startTime)

17 days, 15:38:23.435711
